### Pattern replacement using REGEX

This notebook has been created with the sole purpose of correcting the spelling mistakes that can be found in the attacks dataframe. The reason to create  this additional notebook is mainly practical, as there a lot of pattern to replace, and I have not been able to find of way of optimazing the process

In [51]:
import pandas as pd
import re
import collections
import numpy as np
pd.set_option('display.max_rows',6000)
from datetime import datetime, date

In [2]:
def counts(column):
    '''
    This function returns the value_counts() result of a column as a dictionary sorted by key in ascending order.
    Easier to examine.
    Args: the column of interest in the format of df.column or df["columnd"]
    '''
    s = (column.value_counts())
    dictionary =  s.to_dict()
    return collections.OrderedDict(sorted(dictionary.items()))

In [197]:
data = pd.read_excel("data\global_shark_attacks.xlsx")

In [131]:
data.shape

(6816, 16)

In [147]:
data.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source'],
      dtype='object')

In [202]:
data.columns = data.columns.str.strip()

In [204]:
data_clean = data.drop(columns = {'Case Number', 'Year','Name','Investigator or Source'})

In [150]:
# for categorical values I set Unknown and for date the next value
data_clean.isnull().sum()

Date            303
Type             22
Country          53
Area            481
Location        563
Activity        587
Sex             573
Age            2977
Injury           37
Fatal (Y/N)       0
Time           3503
Species        3082
dtype: int64

In [205]:
data_clean[["Type", "Country", "Area", "Activity", "Injury"
            ,"Sex", "Fatal (Y/N)", "Species", 
      "Location", "Age", "Time"]] = data_clean[["Type", "Country",
                                          "Area", "Activity", "Injury",
                                          "Sex", "Fatal (Y/N)","Species",
                                          "Location", "Age", "Time"]].fillna("Unknown") 

In [206]:
data_clean["Date"] = data_clean["Date"].fillna("1800-01-01")

### Date

In [140]:
def clean_Date(row):
    if isinstance(row['Date'],str):
        try:
            # Extract the first four values to get the year
            year = int(row['Date'][0:4])
            # Extract the two values in the middle to get the month
            month = int(row['Date'][5:7])
            # Extract the two final values to get the day
            day = int(row['Date'][8:10])
            # Create the date value
            return date(year,month,day)
        except: 
            pass     
    else:
        # Remove time from datetime values
        return row['Date'].date()

In [209]:
data_clean['new_date'] = data_clean.apply(lambda row: clean_Date(row),axis=1)

In [ ]:
# data_clean['new_date'] = pd.to_datetime(data_clean['new_date']) --> I need to get rid of the seconds in datetime values

In [210]:
data_clean['new_date'] = pd.to_datetime(data_clean.new_date, errors = 'coerce')

In [211]:
data_clean = data_clean.drop(columns = {"Date"})

### Country

In [212]:
data_clean.Country = data_clean.Country.replace(to_replace = "CEYLON" , value = "SRI LANKA" ) 

In [213]:
data_clean.Country = data_clean.Country.replace(to_replace = r"\?$", value = "", regex = True)

In [214]:
data_clean.Country = data_clean.Country.replace(to_replace = 'MAARTIN', value = 'MARTIN', regex = True)

In [215]:
data_clean.Country = data_clean.Country.replace(to_replace = r'(\w* \/ .*)', value = 'UNKNOWN', regex = True)

In [216]:
data_clean.Country = data_clean.Country.replace(to_replace = 'RED UNKNOWN', value = 'UNKNOWN', regex = True)

In [217]:
data_clean.Country = data_clean.Country.replace(to_replace = 'SOLOMON UNKNOWN', value = 'SOLOMON ISLANDS', regex = True)

In [218]:
data_clean.Country = data_clean.Country.replace(to_replace = ' CEYLON (SRI LANKA)', value = 'SRI LANKA')

In [219]:
data_clean.Country = data_clean.Country.replace(to_replace = 'NEW GUINEA', value = 'PAPUA NEW GUINEA')

In [220]:
data_clean.Country = data_clean.Country.replace(to_replace = 'COLUMBIA', value = 'COLOMBIA')

In [221]:
data_clean.Country = data_clean.Country.replace(to_replace = {'ST HELENA, British overseas territory':'ST HELENA' 
                                                              , 'British Overseas Territory':'Unknown'}, regex=True)

In [222]:
data_clean.Country = data_clean.Country.replace(to_replace = 'JA MAICA', value='JAMAICA')

In [223]:
data_clean.Country = data_clean.Country.replace({' ISLANDS':'',' Island':''}, regex=True)

In [224]:
data_clean.Country = data_clean.Country.replace(to_replace = 'MALDIVE', value='MALDIVES')

In [225]:
data_clean.Country = data_clean.Country.replace(to_replace = 'UNITED ARAB EMIRATES (UAE)', value='UNITED ARAB EMIRATES')

### Second round of modifications to have the proper format for visualzation

In [226]:
data_clean.Country = data_clean.Country.apply(lambda x: x.title().strip() if x != 'USA' else x)

In [227]:
data_clean.Country = data_clean.Country.replace(to_replace = 'Okinawa', value = 'Japan')

In [228]:
data_clean.Country = data_clean.Country.replace(to_replace = 'New Britain', value = 'USA') 

In [229]:
data_clean.Country = data_clean.Country.replace(to_replace = 'Azores', value = 'Portugal')

In [230]:
data_clean.Country = data_clean.Country.replace(to_replace = 'Burma', value = 'Myanmar')

In [231]:
data_clean.Country = data_clean.Country.replace(to_replace = 'Western Samoa', value = 'Samoa')

### Time

In [246]:
data_clean.Time = data_clean.Time.replace(to_replace = ['j','h'],value=':', regex=True)

In [223]:
data_clean.Time = data_clean.Time.replace({'Early ':'','Late ':'','early ':'','late ':'',
                                          'Before ':'','Prior to ':'', 'After ':'',
                                          'Just after ':''},regex=True)

In [224]:
data_clean.Time = data_clean.Time.replace(to_replace = r'(s\--\s\w{2}\w\w{2})', value ='UNKNOWN',regex=True)

In [145]:
data_clean.Time = data_clean.Time.replace(to_replace = r'd{0,4}', value ='d{0,2}:d{0,2}', regex=True)

In [225]:
data_clean.Time = data_clean.Time.replace(to_replace = 'Afternoon', value ='18h30',regex=True)

In [226]:
data_clean.Time = data_clean.Time.replace(to_replace = 'Dusk', value ='07h30',regex=True) 

In [227]:
data_clean.Time = data_clean.Time.replace(to_replace = 'Late afternoon', value ='20h00',regex=True)

In [228]:
data_clean.Time = data_clean.Time.replace(to_replace = 'Evening', value ='20h00')

In [229]:
data_clean.Time = data_clean.Time.replace(to_replace = 'Late afternoon', value ='19h30')

In [230]:
data_clean.Time = data_clean.Time.replace(to_replace = 'Midday', value ='12h00')

In [231]:
data_clean.Time = data_clean.Time.replace(to_replace = 'Midnight', value ='00h00')

In [232]:
data_clean.Time = data_clean.Time.replace(to_replace = 'Morning', value ='08h30', regex=True)

In [233]:
data_clean.Time = data_clean.Time.replace(to_replace = 'Night', value ='21h30')

In [234]:
data_clean.Time = data_clean.Time.replace(to_replace = r'(\/\w*)', value ='', regex = True )

In [235]:
data_clean.Time = data_clean.Time.replace(to_replace = '13h345', value ='13h45')

In [236]:
data_clean.Time = data_clean.Time.replace(to_replace = '15h00j', value ='15h00')

In [237]:
data_clean.Time = data_clean.Time.replace(to_replace = '0830', value ='08h30')

In [238]:
data_clean.Time = data_clean.Time.replace(to_replace = '11h115', value ='11h15')

In [239]:
data_clean.Time = data_clean.Time.replace(to_replace = '06j00', value ='06h00')

In [240]:
data_clean.Time = data_clean.Time.replace(to_replace = r'(\<*)', value ='', regex = True )

In [241]:
data_clean.Time = data.Time.replace(to_replace = r'(\>*)', value ='', regex = True )

In [242]:
data_clean.Time = data_clean.Time.replace(to_replace = 'fall', value ='', regex = True )

In [296]:
data_clean.Time = data_clean.Time.replace(to_replace = 'nig:t', value ='Unknown')

In [287]:
def clean_time(row):
    if isinstance(row['Time'],str) and len(row['Time']) == 5:
        return row['Time']
    else:
        return 'Unknown'

In [330]:
data_clean['Time'] = data_clean.apply(lambda row: clean_time(row),axis=1)

### Activity 

In [186]:
def define_activity(row):
    if "swimming" in row['Activity'].lower():
        return "swimming"
    elif "bath" in row['Activity'].lower() or "standing" in row['Activity'].lower() or "floating" in row['Activity'].lower() or "splash" in row['Activity'].lower() or "play" in row['Activity'].lower():
        return "bathing"
    elif "snorkel" in row['Activity'].lower():
        return "snorkelling"
    elif "kite" in row['Activity'].lower(): 
        return "kite surfing"
    elif "boogie" in row['Activity'].lower():
        return "boogie boarding"
    elif "dive" in row['Activity'].lower() or "diving" in row['Activity'].lower():
        return "diving"
    elif "fishing" in row['Activity'].lower() or "hunting" in row['Activity'].lower() or "Killing" in row['Activity'].lower() or "collecting" in row['Activity'].lower() or "crabbing" in row['Activity'].lower():
        return "fishing"
    elif "kayak" in row['Activity'].lower() or "canoe" in row['Activity'].lower():
        return "kayaking/canoeing"
    elif "surf" in row['Activity'].lower():
        return "surfing"
    elif "cage" in row['Activity'].lower():
        return "cage diving"
    elif "diving" in row['Activity'].lower() or "dived" in row['Activity'].lower() or "dive" in row['Activity'].lower():
        return "diving"
    elif "film" in row['Activity'].lower():
        return "filming"
    elif "capsize" in row['Activity'].lower() or "disaster" in row['Activity'].lower() or "adrift" in row['Activity'].lower() or "wreck" in row['Activity'].lower() or "fell" in row['Activity'].lower():
        return "sea accident"
    elif "air" in row['Activity'].lower():
        return "airplane accident"
    elif "body board" in row['Activity'].lower() or "bodyboarding" in row['Activity'].lower():
        return "body boarding"
    elif "feed" in row['Activity'].lower():
        return "feeding"
    else: 
        return "other"

In [237]:
data_clean['activity'] = data_clean.apply(lambda row: define_activity(row), axis=1)

### Sex

In [23]:
conditions = [
    (data.isin({"Sex":["M"]}))
]

values = ["MALE"]
data['test'] = np.select(conditions, values)

In [322]:
data_clean.Sex = data_clean.Sex.str.rstrip()

In [238]:
data_clean.Sex= data_clean.Sex.replace(to_replace = ['N','M x 2'], value ='M')

In [239]:
data_clean.Sex= data_clean.Sex.replace(to_replace = 'lli', value ='Unknown')

In [240]:
data_clean.Sex= data_clean.Sex.replace(to_replace = '.', value ='Unknown')

### Fatal

In [241]:
data_clean.rename(columns={"Fatal (Y/N)":"Fatal"},inplace=True)

In [242]:
data_clean.Fatal = data_clean.Fatal.replace(to_replace = ['M','n', 'Nq','F'], value = 'N' )
data_clean.Fatal = data_clean.Fatal.replace(to_replace = ['y','Y x 2'], value = 'Y' )
data_clean.Fatal = data_clean.Fatal.replace(to_replace = ['2017','2017.0'], value = 'Unknown' )

In [243]:
data_clean.Fatal = data_clean.Fatal.replace(to_replace = ['y','Y x 2'], value = 'Y' )

In [244]:
data_clean.Fatal = data_clean.Fatal.replace(to_replace = ['2017','2017.0'], value = 'Unknown' )

In [245]:
data_clean.Fatal = data_clean.Fatal.replace(to_replace = 'UNKNOWN', value = 'Unknown' )

In [246]:
list(data_clean.Fatal.unique())

['N', 'Y', 'Unknown']

### Species

In [247]:
data_clean.Species = data_clean.Species.str.lstrip()

In [248]:
data_clean.Species = data_clean.Species.replace(to_replace = r'\d.\d \w|\[.*]', value = '', regex = True)

In [249]:
data_clean.Species = data_clean.Species.replace(to_replace = "\['] |\[] |A |a |m |kg |-lb|\[' ']|\['  ']|\[ ']|\[']|\[. ']|\[.]|\[]|\' ' |\\' |\' |>m|to", value='', regex=True)

In [250]:
sharks = ['White shark','Sandtiger shark','Tiger shark','Lemon shark', 'Blacktip shark', 'Blacktail reef shark',
          'Blacktip reef shark', 'spinner shark','Blue nose shark','Blue pointer', 'Blue pointers', 'Blue shark',
         'Bonitshark','Broadnose sevenguill shark', 'Bronza whaler shark', 'Bronze whaler', 'Bull shark', 
          'Caribbean reef Shark', ' Caribbean reef Shark', 'Carpet shark', 'Copper shark', 'Dog shark', 'Cow shark',
         'Dusky shark', 'Galapagos shark', 'Grey nurse shark', 'Grey reef shark', 'Hammerhead shark',
          'Tiger shark','Wobbegong shark','Grey nurse shark','Nurse shark','Raggedoth shark', 'Mako shark',
         'Bronze whaler shark','Zambesi shark','Sand shark','Lemon shark','Basking shark',
         'Oceanic whitetip shark','Angel shark']

In [251]:
def species_column(value):
    for species in sharks:
        if species.lower() in value.lower():
            return species.lower()
        else: 
            pass
        

In [252]:
data_clean['shark_species'] = list(map(species_column,data_clean.Species))

### Data checkpoint

In [438]:
data_clean.to_csv(r'C:\Users\juanp\proyectos_varios\shark_attacks\data\data_clean.csv' , index = False)

In [44]:
data = pd.read_csv('data\\data_clean.csv')

In [258]:
area = data_clean[data_clean.Country.isin(['Spain', 'USA', 'Brazil', 'Mexico', 
      'Australia', 'South Africa', 'Papua New Guinea','Indonesia','New Zealanda'])]

In [259]:
def areas(row):
    if "Abau Sub" in row['Area']:
        return "Abau Sub District"
    elif "Alicante" in row['Area']:
        return "hardhat diving"
    elif "CÃ¡diz" in row["Area"] or "MÃ¡laga" in row["Area"] or "Granada" in row["Area"] or "Andalucia" in row["Area"]:
        return "Andalusia"
    elif "Baja" in row['Area']:
        return "Baja California"
    elif "Balearics" in row["Area"] or "Majorca" in row["Area"]:
        return "Balearic Islands"
    elif "Boi" in row["Area"]:
        return "Boi Island"
    elif "Bougainville" in row["Area"]:
        return "Bougainville"
    elif "Catalonia" in row["Area"] or "CastellÃ³n" in row['Area'] or "Costa Blanca" in row['Area'] or "Sants-Montjic" in row['Area']:
        return "Catalunya"
    elif "Galica" in row["Area"]:
        return "Galicia"
    elif "Gran Canaria" in row["Area"] or "Grand Canary Islan" in row["Area"] or "Canay Islands" in row['Area']:
        return "Canary Islands"
    elif "Guerro" in row["Area"] or "Guerrrero" in row["Area"]:
        return "Guerrero"
    elif "Eastern Cape" in row["Area"]:
        return "Eastern Cape Province"
    elif "Ibiza Island" in row["Area"]:
        return "Ibiza"
    elif "KZN" in row["Area"] or "KwaZulu" in row["Area"]:
        return "KwaZulu-Natal"
    elif "Madang" in row["Area"]:
        return "Madang"
    elif "Bismarck" in row["Area"]:
        return "Bismarck Archipelago"
    elif "Oro" in row["Area"]:
        return "Oro Province"
    elif "Quinta Roo" in row["Area"]:
        return "Quintana Roo"
    elif "Rio Grande" in row["Area"]:
        return "Rio Grande"
    elif "Cocos" in row["Area"]:
        return "Cocos Islands"
    elif "Vera Cruz" in row["Area"]:
        return "Veracruz"
    elif "Westerm" in row["Area"] or "Western  Australia"  in row["Area"]:
        return "Western Australia"
    elif "Alicante" in row['Area'] in row["Area"]:
        return "Alicante"
    elif "New Ireland" in row['Area']:
        return "New Ireland Province"
    else:
        return row["Area"]
    
area['area'] = area.apply(lambda row: areas(row), axis=1)
    

C:\Users\juanp\AppData\Local\Temp\ipykernel_11120\1477459638.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  area['area'] = area.apply(lambda row: areas(row), axis=1)


In [260]:
x = area.groupby(['Country','area']).agg({'area':'count'})

In [261]:
x = x.rename(columns={'area':'accidents'}).reset_index()

In [14]:
data.to_csv('data\\data_clean.csv')

In [265]:
x.to_csv('data\\areas.csv', index=False)

In [266]:
x.shape

(151, 3)